In [2]:
import sys, glob, os
from pathlib import Path
import numpy as np
from multiprocessing import Pool

## Binarize Training Data

In [37]:
n = 32
cwd = os.getcwd()
prep_train_data_name = "rel_ru_id.txt"


In [38]:
%cd data
!split -n l/{n} {prep_train_data_name} chunk_
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [39]:
def binarize_chunk(f_in_path):
    with open(f_in_path, "r") as f_in, open(
        "data/prep_" + Path(f_in_path).stem, "wb"
    ) as f_out:
        for line in f_in:
            arr = np.fromstring(line, dtype=np.int32, sep=" ")
            f_out.write(
                len(arr).to_bytes(2, byteorder=sys.byteorder, signed=False)
                + arr.tobytes()
            )


In [40]:
with Pool(processes=n) as p:
    p.map(binarize_chunk, glob.glob("data/chunk_*"))


In [41]:
%cd data
!rm chunk_*
!cat prep_chunk_* > ru_small.bin
!rm prep_chunk_*
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [42]:
sample_locs = [0]
with open("data/ru_small.bin", "rb") as f_in:
    while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
        arr = np.frombuffer(f_in.read(count * 4), count=count, dtype=np.int32)
        sample_locs.append(f_in.tell())

sample_locs = np.array(sample_locs[:-1])
np.save("data/sample_locs.npy", sample_locs)

In [3]:
############

arr = np.fromfile("/home/is/armin-sa/Projects/lm/data/ru_small_id.txt", sep=" ", dtype=np.int64)


AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [4]:
np.save("/home/is/armin-sa/Projects/lm/data/ru_small_id.npy", arr)

In [5]:
arr = np.load("/home/is/armin-sa/Projects/lm/data/ru_small_id.npy")
del(arr)